In [ ]:
import sys
print(sys.version)

In [ ]:
pip install openpyxl
pip install torch torchvision torchaudio
pip install dill

In [ ]:
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset

import dill
import torch
import numpy as np
import pandas as pd
import sklearn.metrics as metric

from sklearn import preprocessing
import torch.optim as optim
import torch.nn as nn
import matplotlib.pyplot as plt

In [ ]:
#AutoEncoder 학습을 위한 클래스
class AutoEncoder(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(AutoEncoder, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        self.AutoEncoder = nn.Sequential(
        
        nn.Linear(input_size, hidden_size[0]),
        nn.RReLU(),
        
        nn.Linear(hidden_size[0], output_size),
        nn.RReLU(),
        
        nn.Linear(output_size, hidden_size[0]),
        nn.RReLU(),
        nn.Linear(hidden_size[0], input_size)
        )
        
    def forward(self, inputs):
        output = self.AutoEncoder(inputs)
        
        return output

In [ ]:
# 학습 함수 정의
def train_net(AutoEncoder, data, criterion, epochs, lr, batch_size, optimizer):
    optim = optimizer(AutoEncoder.parameters(), lr = lr)
    data_iter = DataLoader(data, batch_size = batch_size, shuffle = True)
    
    for epoch in range(1, epochs + 1):
        running_loss = 0.0
        for x in data_iter:
            optim.zero_grad()
            output = AutoEncoder(x)
            
            loss = criterion(x, output)
            loss.backward()
            optim.step()
            running_loss += loss.item()
            
        print("epoch: {}, loss: {:2f}".format(epoch, running_loss))
                
    return AutoEncoder

In [ ]:
# 데이터 입력, 전처리 함수
def preprocess_data(file_path):
    welding_data = pd.read_excel(file_path, index_col='idx')

    new_welding_data = welding_data.iloc[:, 5:]

    scaler = preprocessing.MinMaxScaler()
    scaler.fit(new_welding_data)
    scaled_data = scaler.transform(new_welding_data)
    
    split_index = int(len(scaled_data) * 0.8)

    train_data = torch.Tensor(scaled_data[:split_index])
    test_data = torch.Tensor(scaled_data[split_index:])
    
    return train_data, test_data

In [ ]:
train_data, test_data = preprocess_data('파일 이름 넣기')

In [ ]:
def training_data(train_data):
    epoch = 50
    batch_size = 64
    lr = 0.01

    input_size = len(train_data[0])
    hidden_size = [3]
    output_size = 2

    criterion = nn.MSELoss()

    optimizer = torch.optim.Adam
    
    define_AutoEncoder = AutoEncoder(input_size, hidden_size, output_size)
    
    train_AutoEncoder = train_net(define_AutoEncoder, train_data, criterion, epoch, lr, batch_size, optimizer)

    # model_filepath = 'model.pt'
    # save_model(train_AutoEncoder, model_filepath)
    return train_AutoEncoder

In [ ]:
training_data(train_data)

In [ ]:
# 데이터 학습 함수
# epoch - 데이터셋을 학습하는 횟수
# batchSize - 데이터를 몇개의 그룹으로 나눌것인지 정한다.
# lr - 모델 가중치를 조절
# 모델 저장 함수
def save_model(model, filepath):
    torch.save(model, filepath)


def training_data(train_data):
    epoch = 50
    batch_size = 64
    lr = 0.01

    input_size = len(train_data[0])
    hidden_size = [3]
    output_size = 2

    criterion = nn.MSELoss()

    optimizer = torch.optim.Adam
    
    define_AutoEncoder = AutoEncoder(input_size, hidden_size, output_size)
    
    train_AutoEncoder = train_net(define_AutoEncoder, train_data, criterion, epoch, lr, batch_size, optimizer)

    model_filepath = 'model.pt'
    save_model(train_AutoEncoder, model_filepath)
    
training_data(train_data)

In [ ]:
# 저장 데이터 불러오기
# 파일 이름과 저장되있는 주소 값 넣기
def load_model(file_name):
    model = torch.load(file_name, pickle_module = dill)
    return model

model = load_model('./model.pt')

In [ ]:
def save_file(outlier):
    df = pd.DataFrame(outlier, columns=['result'])
    df.to_excel('결과.xlsx', index=True)
    
# 결과출력 함수
def result_data(model, train_data):
    criterion = nn.MSELoss()
    train_loss_chart = []
    for data in train_data:
        output = model(data)
        loss = criterion(output, data)
        train_loss_chart.append(loss.item())
    
    threshold = np.mean(train_loss_chart) + np.std(train_loss_chart)*8
    print("임계값 :", threshold)

    test_loss_chart = []
    for data in train_data:
        output = model(data)
        loss = criterion(output, data)
        test_loss_chart.append(loss.item())


    outlier = list(test_loss_chart >= threshold)
    print("불량 개수 :", outlier.count(True))
     
    return save_file(outlier)
    
def save_file(outlier)
    df = pd.DataFrame(outlier, columns=['result'])
    df.to_excel('결과.xlsx', index=True)

In [ ]:
result_data(model, test_data)